In [1]:
import pandas as pd
import json
from cobra import Model, Reaction, Metabolite
from cobra.io import load_model, write_sbml_model, read_sbml_model, save_json_model, load_matlab_model
from pathlib import Path
from collections import defaultdict
import numpy as np
import re
from scipy.optimize import minimize
from cobra.util.array import create_stoichiometric_matrix

In [77]:
SBMLmodel = Path("iJV803_M_album_BG8_GEM_R_full_version_updated.sbml")
model = read_sbml_model(SBMLmodel)

'' is not a valid SBML 'SId'.


In [78]:
model.optimize

<bound method Model.optimize of <Model  at 0x1c713aaf890>>

In [ ]:
def SLIMEr(model, data, includeTails):
    """
    Python translation of SLIMEr.
    Adapted from the MATLAB function by Benjamín J. Sánchez (2018-05-20).

    This function modifies the given COBRApy model by:
    - Adding biologically relevant lipid backbone and tail metabolites.
    - Generating pseudo-reactions representing lipid assembly (backbones, tails, and full lipids).
    - Replacing any legacy ISA reactions with SLIMEr-compatible formulations. 	ISA = “In Silico Assembled” reaction, a placeholder reaction for lipid synthesis in the old model, replaced by SLIME reactions.
    - Updating stoichiometry to reflect biomass lipid requirements, optionally including tail composition.
    - Removing GAM (growth-associated maintenance) ATP cost if needed for flux balance analyses.

    Parameters:
        model (cobra.Model): A COBRApy metabolic model to be modified.
        data (dict): A dictionary with keys:
            - 'lipidData': Contains backbone metabolite names and optionally their abundances. Must include 'metNames' and will populate 'metIDs' during execution.
            - 'chainData': Contains acyl chain names, formulas, and optionally abundances. Requires keys: 'metNames', 'formulas'.
        includeTails (bool): If True, include acyl chain composition and create the corresponding pseudoreactions. If False, only backbone reactions are modeled.

    Returns:
        tuple:
            - cobra.Model: The updated COBRApy model with added SLIMEr reactions.
            - numpy.ndarray: The stoichiometric matrix (S-matrix) of the updated model.

    Notes:
        - The ATP maintenance component (GAM) is removed to isolate lipid effects.
        - The final merged lipid reaction has the ID 'r_2108', combining tails and backbones into one lipid pool.
    """

    # Add new backbones
    for met in model.metabolites:
        backName = getBackboneName(met.name)
        if backName:
            existing_met = model.metabolites.get_by_id(backName) if backName in model.metabolites else None
            
            model = addLipidSpecies(model, backName, '', False)

            # Add transport reaction to cytoplasm for non-cytoplasmic backbones
            if 'cytoplasm' not in backName:
                cytoName = f"{backName.split('[')[0]}[cytoplasm]"
                existing_cyto_met = model.metabolites.get_by_id(cytoName) if cytoName in model.metabolites else None

                if existing_cyto_met:
                    existing_cyto_met.formula = ''
                else:
                    model = addLipidSpecies(model, cytoName, '', False)

                # Avoid duplicating transport reactions
                transport_rxn_id = f'r_{getNewIndex(model.reactions)}'
                if transport_rxn_id not in model.reactions:
                    trans_rxn = Reaction(transport_rxn_id)
                    trans_rxn.name = f"{backName.split('[')[0]} transport"
                    trans_rxn.lower_bound = 0
                    trans_rxn.upper_bound = 1000
                    trans_rxn.add_metabolites({existing_met: -1, existing_cyto_met: 1})
                    model.add_reactions([trans_rxn])

    # Get backbone metabolite IDs
    metIDs = []
    for metName in data['lipidData']['metNames']:
        cytoName = f"{metName} [cytoplasm]"
        met = model.metabolites.get_by_id(cytoName) if cytoName in model.metabolites else None
        if met:
            metIDs.append(met.id)
    data['lipidData']['metIDs'] = metIDs

    # Add chains
    for i, metName in enumerate(data['chainData']['metNames']):
        fullName = f"{metName} [cytoplasm]"
        formula = data['chainData']['formulas'][i]
        model = addLipidSpecies(model, fullName, formula, not includeTails)

    # Create lipid pseudo-reaction for backbones
    model = addLipidSpecies(model, 'lipid - backbones [cytoplasm]', '', includeTails)
    model = changeLipidComp(model, data['lipidData'])

    # Create lipid pseudo-reaction for tails
    if includeTails:
        model = addLipidSpecies(model, 'lipid - tails [cytoplasm]', '', includeTails)
        model = changeChainComp(model, data['chainData'])

    # Replace ISA reactions with SLIME reactions
    toDelete = []
    for rxn in model.reactions:
        if 'isa ' in rxn.name:
            result = addSLIMErxn(model, rxn.id, None)
            if result is not None:
                model, delete = result
                if delete:
                    toDelete.append(rxn)
        elif rxn.name == 'complex sphingolipid transport':
            toDelete.append(rxn)

    # Add new SLIME reactions for backbones
    for met in model.metabolites:
        backName = getBackboneName(met.name)
        if backName:
            model, _ = addSLIMErxn(model, None, met.id)

    # Change overall lipid pseudo-reaction
    back_met = model.metabolites.get_by_id('lipid - backbones [cytoplasm]')
    lipid_met = model.metabolites.get_by_id('lipid [cytoplasm]')

    if includeTails:
        tail_met = model.metabolites.get_by_id('lipid - tails [cytoplasm]')
        mets = [tail_met, back_met, lipid_met]
        coeffs = [-1, -1, 1]
    else:
        mets = [back_met, lipid_met]
        coeffs = [-1, 1]

    lipid_rxn = Reaction('r_2108')
    lipid_rxn.name = 'lipid pseudoreaction - merge'
    lipid_rxn.lower_bound = 0
    lipid_rxn.upper_bound = 1000
    lipid_rxn.add_metabolites({mets[i]: coeffs[i] for i in range(len(mets))})
    model.add_reactions([lipid_rxn])

    # Remove unused ISA reactions properly
    model.remove_reactions(toDelete)

    # Remove GAM requirement
    GAM = 0.0
    bio_rxn = model.reactions.get_by_id('biomass pseudoreaction')
    ATP = model.metabolites.get_by_id('ATP [cytoplasm]')
    H2O = model.metabolites.get_by_id('H2O [cytoplasm]')
    ADP = model.metabolites.get_by_id('ADP [cytoplasm]')
    H = model.metabolites.get_by_id('H+ [cytoplasm]')
    P = model.metabolites.get_by_id('phosphate [cytoplasm]')

    bio_rxn.add_metabolites({ATP: -GAM, H2O: -GAM, ADP: GAM, H: GAM, P: GAM})

    S = extractModelStoichMatrix(model)

    return model, S 


In [ ]:
def addSLIMErxn(model, rxnID, specID):
    """
    Adds a SLIME pseudo-reaction to a metabolic model for a specific lipid species.
    It either replaces a given ISA reaction (if `specID` is empty) or generates a new SLIME reaction for a specified 
    lipid species. It ensures that the reaction is mass-balanced by computing the molecular formula for 
    the backbone metabolite based on the species and tails.

    Parameters:
        model (cobra.Model): The COBRApy model to be modified.
        rxnID (str): The ID of the existing reaction to replace (if any). If empty, a new ID will be generated.
        specID (str): The ID of the lipid species to use in the reaction. If empty, it is inferred from the ISA reaction.

    Returns:
        tuple:
            - cobra.Model: The updated COBRApy model with the SLIME reaction added.
            - bool: True if the original ISA reaction should be deleted (e.g., it couldn’t be converted), False otherwise.

    Notes:
        - Automatically determines the lipid backbone name using `getBackboneName()` and checks if it exists in the model.
        - Tail metabolites must be present in the model and have names ending with `' chain [cytoplasm]'`.
        - The backbone metabolite's formula is computed to ensure elemental balancing using the `getStoichFromFormula()` helper.
    """
    toDelete = False
    
    if specID == '':  # Case 1: Existing ISA reaction
        reaction = model.reactions.get_by_id(rxnID)
        specID = list(reaction.metabolites.keys())[0].id
    
    if rxnID == '':  # Case 2: New ISA reaction
        rxnID = 'r_' + getNewIndex([r.id for r in model.reactions])
    
    specMet = model.metabolites.get_by_id(specID)
    specName = specMet.name
    rxnName = specName + ' SLIME rxn'
    
    backName = getBackboneName(specName)
    if backName is None:
        print('Removing unrecognized ISA reaction')
        return model, True
    
    backMet = next((m for m in model.metabolites if m.name.startswith(backName + ' [')), None)
    if not backMet:
        print(f'Backbone metabolite {backName} not found')
        return model, True
    
    specFormula = specMet.formula
    specMW = getMWfromFormula(specFormula)
    
    # Determine fatty acid tails for different lipid classes
    tailsRxn = []
    if backName in ['phosphatidylserine', 'phosphatidylethanolamine', 'phosphatidate', 'phosphatidylglycerol', 'cardiolipin']:
        tailsRxn = ['C' + part for part in specName.split('(')[-1].replace(')', '').split(', ')]

    # Need to decide our fatty acid names, ie: palmitate C16:0? myristoyl acid C14:0? iso-hepta-1-methyl-decanoyl as C15:0 ?? 
    # Recall that according to FAMES on BG8 has [C10:0, C12:0, C14:0, C15:0. C16:0, C16:ln9?, C16:ln7?, C16:ln6?, C16:ln5?, C16unknown1, C16:2, C16:3, C18:0, C18:ln9, C18:ln7 ]
    elif backName in ['fatty acid', 'ergosterol ester', 'long-chain base', 'long-chain base phosphate']:
        species_map = {
            'myristate': 'C14:0', 'palmitate': 'C16:0', 'stearate': 'C18:0',
            'oleate': 'C18:1', 'sphinganine': 'C18:0', 'phytosphingosine': 'C18:0'
        }
        tailsRxn = [species_map.get(specName)]
    
    tailIDs, tailsFormulas, tailsMWs = [], [], []
    for met in model.metabolites:
        if ' chain [cytoplasm]' in met.name:
            tailIDs.append(met.id)
            tailsFormulas.append(met.formula)
            tailsMWs.append(getMWfromFormula(met.formula))
    
    tailCoeffs = np.zeros(len(tailIDs))
    prodFormulas = []
    
    for tail in tailsRxn:
        tailName = tail + ' chain [cytoplasm]'
        try:
            idx = next(i for i, m in enumerate(model.metabolites) if m.name == tailName)
            tailCoeffs[idx] += 1
            prodFormulas.append(tailsFormulas[idx])
        except StopIteration:
            raise ValueError(f"Tail metabolite '{tailName}' not found in the model.")
    
    # Assign molecular formula to backbone (balance SLIME rxn)
    backFormula = ''
    for elem in ['C', 'H', 'N', 'O', 'P', 'S']:
        Nin = getStoichFromFormula(specFormula, elem)
        Nout = sum(getStoichFromFormula(f, elem) for f in prodFormulas)
        diff = Nin - Nout
        if diff > 0:
            backFormula += f'{elem}{diff}' if diff > 1 else elem
    
    backMet.formula = backFormula  # Assign to backbone metabolite
    
    # Add SLIME reaction
    reaction = Reaction(rxnID)
    reaction.name = rxnName
    reaction.lower_bound = 0
    reaction.upper_bound = 1000
    
    reaction.add_metabolites({
        specMet: -1,
        backMet:specMW
    })
    
    for tailID, coeff in zip(tailIDs, tailCoeffs):
        if coeff != 0:
            tailMet = model.metabolites.get_by_id(tailID)
            reaction.add_metabolites({tailMet: coeff})
    
    model.add_reactions([reaction])
    
    return model, toDelete


In [ ]:
def getBackboneName(spec_name):
    """
    Infers the generic lipid backbone name from a lipid species name.
    It parses a lipid species name (e.g., "phosphatidylethanolamine (C16:0, C18:1)") and returns a corresponding backbone identifier 
    (e.g., "phosphatidylethanolamine"). This is used to associate specific lipid species with generalized pseudometabolites in SLIMEr reactions.

    Parameters:
        spec_name (str): The full name of the lipid species.

    Returns:
        str: The inferred backbone name. If no match is found, returns an empty string.

    Logic:
        - Group 1 backbones are matched if the input starts with the backbone name, has additional details (e.g., tails), and is not just the plain name or a phosphate variant.
        - Group 2 maps specific common fatty acids (e.g., "palmitate", "oleate") to the generic backbone "fatty acid".

    Notes:
        - This function assumes consistent formatting for metabolite names.
        - The returned name should be used to match against generic pseudometabolites in the model.
    """

    back_name = ""

     # Group 1: keep generic name if conditions match
    group1 = [
        "phosphatidylserine",
        "phosphatidylethanolamine",
        "phosphatidate",
        "phosphatidylglycerol",
        "cardiolipin"
    ]

    for name in group1:
        if spec_name.lower().startswith(name.lower()) and spec_name.lower() != name.lower() and "phosphate" not in spec_name.lower():
            back_name = name 
            break

    # Group 2: replace specific name with generic backbone name
    if not back_name:
        group2 = {
            "palmitate": "fatty acid",
            "palmitoleate": "fatty acid",
            "stearate": "fatty acid",
            "oleate": "fatty acid",
        }

        if spec_name in group2:
            back_name = group2[spec_name]

    return back_name

In [ ]:
def addLipidSpecies(model, metName, metFormula, exchange):
    """
    Adds a lipid metabolite to the model, with an optional exchange reaction.
    This function simplifies metabolite creation by bypassing compartment handling and assigning 
    a unique ID based on model contents. If `exchange` is True, an exchange reaction is also added.

    Parameters:
        model (cobra.Model): The COBRApy model to modify.
        metName (str): The name of the new metabolite (should include compartment if needed).
        metFormula (str): The chemical formula of the metabolite (can be empty if unknown).
        exchange (bool): Whether to create an associated exchange reaction (forward, irreversible).

    Returns:
        cobra.Model: The updated model with the new metabolite (and optionally an exchange reaction).

    Notes:
        - Metabolite IDs follow the convention 's_{index}' using `getNewIndex()` for uniqueness.
        - Exchange reaction IDs follow the 'r_{index}' format and are named after the metabolite.
        - Exchange reactions are added with bounds [0, 1000] to allow unlimited export.
    """

    newID = getNewIndex(model.metabolites)
    metID = f's_{newID}'

    new_met = Metabolite(
        id=metID,
        name=metName,
        formula=metFormula
    )
    model.add_metabolites([new_met])

    # Add exchange reaction if needed
    if exchange:
        newID = getNewIndex(model.reactions)
        rxnName = f'{metName} exchange'

        exchange_rxn = Reaction(
            id=f'r_{newID}',
            name=rxnName,
            lower_bound=0,
            upper_bound=1000
        )
        exchange_rxn.add_metabolites({new_met: -1})
        model.add_reactions([exchange_rxn])

        print(f'{exchange_rxn.id}: {rxnName} - {metID} ->')

    return model


In [ ]:
def changeLipidComp(model, lipidData):
    """
    Updates or creates the lipid backbone pseudo-reaction based on lipid composition data.
    It constructs a pseudo-reaction where each backbone metabolite contributes to a single product 
    representing the lipid pool. If a 'lipid - backbones' metabolite is found, it creates a new 
    reaction; otherwise, it modifies an existing one with a fallback metabolite ID.

    Parameters:
        model (cobra.Model): The COBRApy metabolic model to modify.
        lipidData (dict): A dictionary with keys:
            - 'metIDs': List of COBRApy metabolite IDs for lipid backbones.
            - 'abundance': List of abundance values (float), must align with `metIDs` order.

    Returns:
        cobra.Model: The updated model with a new or modified lipid backbone pseudo-reaction.

    Notes:
        - Reaction stoichiometry is constructed with negative abundances for each input metabolite and a +1 coefficient for the output metabolite representing the lipid pool.
        - If a metabolite named 'lipid - backbones' exists in the model, a new reaction is created with a unique ID. Otherwise, a fallback reaction 'r_2108' is used with a hardcoded metabolite ID ('s_1096[c]').
        - Assumes that all metabolite IDs provided exist in the model.
    """

    # Detect if the model has a metabolite for backbones
    metID = None
    for met in model.metabolites:
        if met.name == 'lipid - backbones':
            metID = met.id
            break

    if metID:
        # Create new pseudo-reaction for backbones
        newID = getNewIndex(model.reactions)
        rxnID = f'r_{newID}'
        rxnName = 'lipid pseudoreaction - backbone'
    else:
        # Use fallback 
        rxnID = 'r_2108'
        rxnName = 'lipid pseudoreaction'
        metID = 's_1096[c]'  # Assume this ID exists in the model

    # Create lipid pseudo-reaction for backbones (or modify normal one)
    metaboliteList = lipidData['metIDs'] + [metID]
    stoichCoeffList = [-x for x in lipidData['abundance']] + [1]

    stoichDict = {}
    for i in range(len(metaboliteList)):
        met = model.metabolites.get_by_id(metaboliteList[i])
        coeff = stoichCoeffList[i]
        stoichDict[met] = coeff

    # Create and add the reaction
    rxn = Reaction(id=rxnID, name=rxnName, lower_bound=0, upper_bound=1000)
    rxn.add_metabolites(stoichDict)
    model.add_reactions([rxn])

    return model

In [ ]:
def changeChainComp(model, chainData):
    """
    Adds a lipid pseudoreaction to the model representing tail (acyl chain) composition.
    This function creates a pseudo-reaction that consolidates all tail metabolites into a single output metabolite, 'lipid - tails'. Each tail metabolite contributes according to 
    its specified abundance, simulating the average tail composition in biomass.

    Parameters:
        model (cobra.Model): The COBRApy model to update.
        chainData (dict): Dictionary containing tail metabolite info:
            - 'metNames' (list of str): Names of acyl chain metabolites (should match exactly with names in the model).
            - 'abundance' (list of float): Corresponding abundance values for each tail metabolite.

    Returns:
        cobra.Model: The model updated with a new reaction simulating average tail contribution.

    Notes:
        - The new reaction is created with a unique ID (e.g., 'r_XXXX') and named 'lipid pseudoreaction - tail'.
        - Stoichiometry is set such that each tail metabolite is consumed (negative coefficient), and 'lipid - tails' is produced (+1).
        - Uses a dictionary lookup for efficient metabolite access.
        - Abundances are expected to sum to 1.0 for meaningful biological interpretation.
    """
    # Identify metabolite IDs for the tails
    metabolite_lookup = {met.name: met for met in model.metabolites}  # only one pass through model.metabolites

    tailIDs = []
    for metName in chainData['metNames']:
        tailName = metName
        if tailName in metabolite_lookup:
            tailIDs.append(metabolite_lookup[tailName])
        else:
            raise ValueError(f"Metabolite {tailName} not found in model.")
    
    # Generate new reaction ID and name
    newID = getNewIndex([r.id for r in model.reactions])
    rxnID = 'r_' + newID
    rxnName = 'lipid pseudoreaction - tail'
    
    # Find the main 'lipid - tails' metabolite
    if 'lipid - tails' not in metabolite_lookup:
        raise ValueError("Metabolite 'lipid - tails' not found in model.")
    tailMet = metabolite_lookup['lipid - tails']
    
    # Create the new reaction
    reaction = cobra.Reaction(rxnID)
    reaction.name = rxnName
    reaction.lower_bound = 0
    reaction.upper_bound = 1000
    
    # Define metabolite stoichiometry
    # IE: tailIDs = ['C16:0', 'C18:0']
    # chainData['abundance'] = [0.3, 0.7]
    # [('C16:0', 0.3), ('C18:0', 0.7)]
    metabolites_dict = {met: -abundance for met, abundance in zip(tailIDs, chainData['abundance'])}
    metabolites_dict[tailMet] = 1  # This ensures that lipid - tails is produced with a coefficient of +1.
    
    reaction.add_metabolites(metabolites_dict)
    model.add_reactions([reaction])

    return model

In [ ]:
def changeOtherComp(model, data):
    """
    Modifies the biomass composition of a COBRApy model by adjusting the stoichiometry of protein, RNA, carbohydrate, DNA, and other components based on user-specified abundances.
    It updates stoichiometric coefficients in pseudoreactions that make up the biomass objective function and recalculates polymerization-associated maintenance energy (GAMpol).

    Parameters:
        model (cobra.Model): The metabolic model to be modified.
        data (dict): A dictionary containing 'otherData', where:
            - 'metIDs' (list of str): A list of component identifiers to be updated, e.g., 'protein', 'RNA', or specific metabolite IDs.
            - 'abundance' (list of float): Corresponding abundances for each metabolite/component listed in `metIDs`.

    Returns:
        tuple:
            - cobra.Model: The updated model with new biomass composition coefficients.
            - float: Estimated polymerization-associated maintenance energy (GAMpol) in mmol ATP/gDW.

    Notes:
        - The `comps` list defines the molecular weights and biomass class (P: protein, C: carbohydrate, R: RNA, D: DNA, N: other) of known biomass components.
        - Updates pseudoreactions for protein, RNA, DNA, carbohydrates, and other miscellaneous components by:
            1. Scaling existing stoichiometric entries for protein and RNA.
            2. Overwriting stoichiometries for carbohydrates, DNA, and unknown components based on provided mass fractions.
        - Rebalances the carbohydrate pseudoreaction to ensure total biomass composition sums to 1 g/gDW.
        - GAMpol is calculated using empirical constants based on macromolecule class contributions.
    """

    otherData = data['otherData']

    # Define the components list for biomass NEED THIS FOR OURS
    comps = [
        ('s_0404[c]', 89.09, 'P'), ('s_0542[c]', 121.16, 'P'), ('s_0432[c]', 133.11, 'P'),
        ('s_0748[c]', 147.13, 'P'), ('s_1314[c]', 165.19, 'P'), ('s_0757[c]', 75.07, 'P'),
        ('s_0832[c]', 155.15, 'P'), ('s_0847[c]', 131.17, 'P'), ('s_1099[c]', 146.19, 'P'),
        ('s_1077[c]', 131.17, 'P'), ('s_1148[c]', 149.21, 'P'), ('s_0430[c]', 132.12, 'P'),
        ('s_1379[c]', 115.13, 'P'), ('s_0747[c]', 146.14, 'P'), ('s_0428[c]', 174.2, 'P'),
        ('s_1428[c]', 105.09, 'P'), ('s_1491[c]', 119.12, 'P'), ('s_1561[c]', 117.15, 'P'),
        ('s_1527[c]', 204.23, 'P'), ('s_1533[c]', 181.19, 'P'), ('s_0001[ce]', 180.16, 'C'),
        ('s_0004[ce]', 180.16, 'C'), ('s_0509[c]', 221.21, 'C'), ('s_0773[c]', 180.16, 'C'),
        ('s_1107[c]', 180.16, 'C'), ('s_1520[c]', 342.296, 'C'), ('s_0423[c]', 347.22, 'R'),
        ('s_0526[c]', 323.2, 'R'), ('s_0782[c]', 363.22, 'R'), ('s_1545[c]', 324.18, 'R'),
        ('s_0584[c]', 331.22, 'D'), ('s_0589[c]', 307.2, 'D'), ('s_0615[c]', 345.21, 'D'),
        ('s_0649[c]', 322.21, 'D'), ('s_3714[c]', 852.83, 'N'), ('s_1405[c]', 376.36, 'N'),
        ('s_1467[c]', 96.06, 'N')
    ]

    # Get initial component masses
    X, P, C, R, D, _ = sumBioMass(model, comps)

    protPos = model.rxnNames.index('protein pseudoreaction')
    rnaPos = model.rxnNames.index('RNA pseudoreaction')

    for i, metID in enumerate(otherData['metIDs']):
        abundance = otherData['abundance'][i]

        if metID == 'protein':
            fP = abundance / P
            isAA = ['tRNA' in name for name in model.metNames]
            for j, aa in enumerate(isAA):
                if aa:
                    model.S[j, protPos] *= fP

        elif metID == 'RNA':
            fR = abundance / R
            nucs = [c[0] for c in comps if c[2] == 'R']
            for nuc in nucs:
                modelPos = model.mets.index(nuc)
                model.S[modelPos, rnaPos] *= fR

        else:
            modelPos = model.mets.index(metID)
            comp = next(c for c in comps if c[0] == metID)

            if comp[2] == 'C':
                rxnPos = model.rxnNames.index('carbohydrate pseudoreaction')
                MW = comp[1] - 18
            elif comp[2] == 'D':
                rxnPos = model.rxnNames.index('DNA pseudoreaction')
                MW = comp[1] - 18
            elif comp[2] == 'N':
                rxnPos = model.rxnNames.index('biomass pseudoreaction')
                MW = comp[1]

            model.S[modelPos, rxnPos] = -abundance / MW * 1000

    # Recompute biomass and balance with sugars
    X, _, C, _, _, _ = sumBioMass(model, comps)
    delta = X - 1  # Difference to balance

    # Balance with all carbohydrate components
    mets = [c[0] for c in comps if c[2] == 'C']
    massPre = C
    massPost = massPre - delta
    fC = massPost / massPre

    carbPos = model.rxnNames.index('carbohydrate pseudoreaction')
    for met in mets:
        modelPos = model.mets.index(met)
        model.S[modelPos, carbPos] *= fC

    # Estimate polymerization-associated maintenance energy
    _, P, C, R, D, _ = sumBioMass(model, comps)
    GAMpol = P * 37.7 + C * 12.8 + R * 26.0 + D * 26.0 

    return model, GAMpol

In [ ]:
def scaleAbundancesInModel(model, data, scaling):
    """
    Optimizes and applies a global scaling factor to lipid abundances to minimize unused lipids in the metabolic model.
    It finds the best scaling coefficient `k` such that lipid-associated exchange flux is minimized, ensuring that backbone or tail components are properly integrated into biomass with minimal excess.

    Parameters:
        model (cobra.Model): The COBRApy model to be adjusted.
        data (dict): Input data required for model simulation and scaling. Must contain:
            - 'fluxData': Required for simulating growth.
        scaling (str): Either 'backbones' or 'tails', determining which lipid component is being scaled.

    Returns:
        tuple:
            - cobra.Model: The updated model with adjusted lipid stoichiometry.
            - float: The applied scaling factor `k`.

    Notes:
        - Uses `scipy.optimize.minimize` with Nelder-Mead to find optimal `k`.
        - Computes a feasible `k` range using custom objective functions to bound behavior.
        - The average of the range is used to adjust the model.
        - Calls `adjustModel()` to apply the new stoichiometry.
    """
    # Initial guess for k
    k0 = 1

    # Find optimal scaling factor using fminsearch equivalent (scipy's minimize)
    result = minimize(lambda k: unusedLipid(k[0], model, data, scaling), [k0], method='Nelder-Mead')
    #Find the optimal scaling factor kOpt. This ensures minimal unused lipid accumulation in the model.
    kOpt = result.x[0]

    # Find optimality range
    krange = [0, 0]
    result1 = minimize(lambda k: +minScaling(k[0], model, data, scaling, kOpt), [kOpt], method='Nelder-Mead')
    krange[0] = result1.x[0]
    
    result2 = minimize(lambda k: -minScaling(k[0], model, data, scaling, kOpt), [kOpt], method='Nelder-Mead')
    krange[1] = result2.x[0]

    print(f"Optimality range: k = [ {krange[0]} , {krange[1]} ]")

    if krange[0] == krange[1]:
        raise ValueError('Could not find an optimality range!')

    # Scale using the average of the optimality range
    k = np.mean(krange)
    model = adjustModel(model, k, True, scaling)
    print(f"Scaled {scaling[:-1]} data in model: k = {k}")

    return model, k


def unusedLipid(k, model, data, scaling):
    """
    Objective function for optimization

    Parameters:
        k (float): Scaling factor to apply.
        model (cobra.Model): The COBRApy model.
        data (dict): Data containing 'fluxData' used for growth simulation.
        scaling (str): Either 'tails' or 'backbones'.

    Returns:
        float: Combined exchange flux of unused lipid tails and backbones (minimized during optimization).
    """

    # Adjust stoichiometry using scaling factor
    model = adjustModel(model, k, False, scaling)

    # Simulate growth
    try:
        sol = simulateGrowth(model, data['fluxData'])
    except:
        sol = {'x': np.ones(len(model.reactions))}

    # Objective function: unused tails or backbones
    exchange_tails = getReactionFlux(model, sol, 'lipid - tails exchange')
    exchange_backs = getReactionFlux(model, sol, 'lipid - backbones exchange')
    exchange = exchange_tails + exchange_backs

    print(f"Scaling abundance data: k = {k} -> exchange = {exchange}")

    return exchange


def minScaling(k, model, data, scaling, kOpt):
    """
    Returns a scaling factor `k` if it leads to feasible model simulation and non-zero NGAM flux.
    Used to bound the optimality range around `kOpt`.

    Parameters:
        k (float): Scaling factor candidate.
        model (cobra.Model): The COBRApy model.
        data (dict): Simulation input data.
        scaling (str): Scaling target ('backbones' or 'tails').
        kOpt (float): Previously determined optimal scaling value.

    Returns:
        float: Valid scaling factor `k`, or falls back to `kOpt` if simulation fails.
    """

    # Adjust stoichiometry with k
    model = adjustModel(model, k, True, scaling)

    # Simulate growth and get NGAM 
    try:
        sol = simulateGrowth(model, data['fluxData'])
        posNGAM = getReactionIndex(model, 'non-growth associated maintenance reaction')
        maintenance = sol['x'][posNGAM]
        print(f"Finding scaling range: k = {k} -> Maintenance = {maintenance}")
    except:
        print(f"Finding scaling range: k = {k} -> Maintenance = 0")
        k = kOpt  # Any unfeasible simulation returns the original value

    return k


def getReactionFlux(model, sol, rxn_name):
    """
    Gets the flux value for a specified reaction from the solution.

    Parameters:
        model (cobra.Model): The COBRApy model.
        sol (dict): Dictionary with fluxes (e.g., from `simulateGrowth()`).
        rxn_name (str): Name of the reaction to retrieve flux for.

    Returns:
        float: Flux value, or 0 if the reaction is not found or simulation failed.
    """
    try:
        rxn_index = getReactionIndex(model, rxn_name)
        return sol['x'][rxn_index]
    except:
        return 0


def getReactionIndex(model, rxn_name):
    """
    Returns the index of a reaction in the model's reaction list by name.

    Parameters:
        model (cobra.Model): The COBRApy model.
        rxn_name (str): Reaction name to search for.

    Returns:
        int: Index of the reaction.
    """
    for i, rxn in enumerate(model.reactions):
        if rxn.name == rxn_name:
            return i
    raise ValueError(f"Reaction '{rxn_name}' not found in model.")


def simulateGrowth(model, fluxData):
    """
    Simulates model growth using FBA.

    Parameters:
        model (cobra.Model): The COBRApy model.
        fluxData (any): Fluxdata input.

    Returns:
        dict: Dictionary containing the flux solution as `sol['x']`.
    """
    solution = model.optimize()  
    return {'x': solution.fluxes.values} 
    pass

In [ ]:
def getNewIndex(ids):
    """
    Generates a new numeric index string based on the highest existing ID in a list.
    This function parses a list of COBRApy metabolite or reaction IDs (typically in the format 's_###' or 'r_###'),
    extracts their numeric components, and returns the next available integer as a string. It is used to 
    assign unique IDs for new reactions or metabolites added to the model.

    Parameters:
        ids (list of str): A list of existing reaction or metabolite IDs from the model.

    Returns:
        str: A string representing the next available numeric ID (e.g., '104' if the max was 103).
    """
    # Extract numeric parts from the IDs
    numeric_ids = []
    for id_str in ids:
        # Extract all numeric substrings (assuming IDs follow the "s_###" or "r_###" format)
        numbers = re.findall(r'\d+', id_str)
        if numbers:
            numeric_ids.extend([int(n) for n in numbers])

    # Find the maximum number and increment it
    if numeric_ids:
        new_id = max(numeric_ids) + 1
    else:
        new_id = 1  # Start from 1 if no numeric IDs exist

    return str(new_id)

In [ ]:
def modelsFromData(model, data, scaling):
    """
    Builds two versions of a genome-scale metabolic model using experimental lipid data and SLIMEr constraints.
    This function applies a pipeline that incorporates lipid and tail composition data, adjusts biomass
    component abundances, and scales lipid fluxes to minimize unused byproducts. It returns two models:
    one with only corrected lipid composition, and another with both lipid and chain-length constraints 
    applied.

    Parameters:
        model (cobra.Model): The original COBRApy model.
        data (dict): Experimental data including:
            - 'lipidData': Lipid backbone names, IDs, and abundances.
            - 'chainData': Fatty acid tail names, formulas, and abundances.
            - 'otherData': Protein, RNA, carbohydrate, DNA, and other biomass component abundances.
            - 'fluxData': Required for simulating growth in scaling steps.
        scaling (str): Either 'backbones' or 'tails', determines which component is scaled during optimization.

    Returns:
        tuple:
            - model_corrComp (cobra.Model): Model with updated lipid composition only.
            - model_SLIMEr (cobra.Model): Fully constrained model with both lipid and tail constraints.
            - k (float): Optimal scaling factor applied to lipid abundance data.
            - GAMpol (float): Estimated polymerization-associated maintenance energy (in mmol ATP/gDW).
    """

    # Model with lipid composition corrected
    model_corrComp = SLIMEr(model, data, constrain_chain_lengths=False)

    # Model with both lipid composition and chain length constrained
    model_SLIMEr = SLIMEr(model, data, constrain_chain_lengths=True)

    # Correct the biomass composition with data for both models
    model_corrComp, _ = changeOtherComp(model_corrComp, data)
    model_SLIMEr, _ = changeOtherComp(model_SLIMEr, data)

    # Make abundances of backbones & chains consistent
    model_SLIMEr, k = scaleAbundancesInModel(model_SLIMEr, data, scaling)

    # Adjust model_corrComp using the same scaling factor k
    model_corrComp = adjustModel(model_corrComp, k, constrain_chain_lengths=False, scaling=scaling)

    # Correct biomass composition again to account for updated lipid content
    model_corrComp, _ = changeOtherComp(model_corrComp, data)
    model_SLIMEr, GAMpol = changeOtherComp(model_SLIMEr, data)

    return model_corrComp, model_SLIMEr, k, GAMpol

In [ ]:
"""
Creates multiple condition-specific metabolic models using lipid composition data and SLIMEr constraints.
It generates two types of models for each growth condition: a composition-corrected model (corrComp) and a SLIMEr-constrained model (SLIMEr), using experimental 
lipidomic data.
"""

# WE NEED TO MAKE THIS BASED ON OUR DATA BUT I DONT UNDERSTAND WHAT THEY ARE DOING EXACTLY
# ONE SET OF DATA SEEMS TO BE FOR SLIME, ONE SET SEEMS TO BE FOR STUDIES THAT VALIDATE ??
# WOULD WE DO THIS BECAUSE CATHERINES PAPER HAS 4 CONDITIONS THEN TECHNICALLY WE HAVE 6 WITH OUR VALIDATIONS ??

model = io.read_sbml_model('yeast780.xml')  # or 'yeast780.json', depending on format

# Create 2 models for each of the 10 conditions in the stress dataset
# One model is a "corrected" (corrComp) model and the other is the model adjusted for slime (SLIMEr)
model_corrComp = [None] * 10
model_SLIMEr = [None] * 10
GAMpol = np.zeros(10)
k = np.zeros(10)

for i in range(10):
    data = readLahtveeData(i + 1)  
    model_corrComp[i], model_SLIMEr[i], k[i], GAMpol[i] = modelsFromData(model, data, 'backbones')

# Create 2 models for each of the validation studies
# One model is a "corrected" (corrComp) model and the other is the model adjusted for slime (SLIMEr)
model_corrComp_val = [None] * 8
model_SLIMEr_val = [None] * 8

for i in range(8):
    data = readEsjingData(i + 1)  
    data = convertEsjingData(data, model, True)
    model_corrComp_val[i], model_SLIMEr_val[i], _, _ = modelsFromData(model, data, 'tails')

In [ ]:
def adjustModel(model, k, block, scaling):
    """
    Scales stoichiometric coefficients in a lipid-related pseudo-reaction and optionally blocks related exchanges.
    This function modifies the stoichiometric matrix `S` for either the lipid backbone or tail pseudo-reaction 
    by applying a scaling factor `k`. 
    Parameters:
        model (cobra.Model): The COBRApy model to be adjusted.
        k (float): The scaling factor to apply to the stoichiometric coefficients.
        block (bool): If True, sets the bounds of 'lipid - tails exchange' and 'lipid - backbones exchange' to (0, 0).
        scaling (str): Must be either 'backbones' or 'tails'. Determines which pseudo-reaction is scaled.

    Returns:
        cobra.Model: The updated model with adjusted stoichiometry and optionally blocked exchanges.

    Notes:
        - Only reactants (i.e., metabolites with negative stoichiometric coefficients) are rescaled.
    """
    # Extract model data because they use a matrix for scaling
    S, rxnNames, rxns = extractModelStoichMatrix(model)

    # Block exchange of tails and backbones
    if block:
        posT = np.where(np.array(rxnNames) == 'lipid - tails exchange')[0]
        posB = np.where(np.array(rxnNames) == 'lipid - backbones exchange')[0]

        if posT.size > 0:
            rxn = model.reactions.get_by_id(rxns[posT[0]])
            rxn.bounds = (0, 0)

        if posB.size > 0:
            rxn = model.reactions.get_by_id(rxns[posB[0]])
            rxn.bounds = (0, 0)

    if scaling == 'backbones':
        rxnName = 'lipid pseudoreaction - backbone'
    elif scaling == 'tails':
        rxnName = 'lipid pseudoreaction - tail'
    else:
        raise ValueError(f"Invalid scaling type: {scaling}")

    scaleRxnPos = np.where(np.array(rxnNames) == rxnName)[0]
    if scaleRxnPos.size == 0:
        raise ValueError(f"Reaction '{rxnName}' not found in model.")

    scaleRxnPos = scaleRxnPos[0]

    scaleMets = np.where(S[:, scaleRxnPos] < 0)[0]

    # Rescale stoichiometric coefficients
    for metPos in scaleMets:
        S[metPos, scaleRxnPos] *= k

    return model 

In [ ]:
def extractModelStoichMatrix(model):
    """
    Extracts the stoichiometric matrix and reaction metadata from a COBRApy model.

    Parameters:
        model (cobra.Model): The COBRApy model from which to extract stoichiometry and reaction metadata.

    Returns:
        tuple:
            - numpy.ndarray: Dense stoichiometric matrix (metabolites x reactions).
            - list of str: Names of all reactions in the model.
            - list of str: IDs of all reactions in the model.

    Notes:
        - This function assumes reaction ordering is consistent between `rxnNames`, `rxns`, and the matrix columns.
    """
    S = create_stoichiometric_matrix(model, array_type='dense')
    rxnNames = [rxn.name for rxn in model.reactions]  
    rxns = [rxn.id for rxn in model.reactions] 
    
    return S, rxnNames, rxns

In [ ]:
def sumBioMass(model, comps):
    """
    Calculates the contribution of major macromolecular classes to total biomass.
    This function quantifies the g/gDW contribution of proteins, carbohydrates, RNA, DNA, and lipids
    based on their associated pseudoreactions and stoichiometry in the biomass reaction. It returns 
    a breakdown of these components, as well as the total biomass excluding lipids (`X`).

    Parameters:
        model (cobra.Model): The COBRApy model with defined biomass and pseudoreactions.
        comps (list of tuple): List of component tuples, where each tuple contains:
            - metabolite ID (str)
            - molecular weight (float, in g/mol)
            - component type (str): One of ['P', 'C', 'R', 'D', 'L'] for protein, carbohydrate, RNA, DNA, lipid.

    Returns:
        tuple:
            - X (float): Biomass fraction excluding lipids [g/gDW].
            - P (float): Protein fraction [g/gDW].
            - C (float): Carbohydrate fraction [g/gDW].
            - R (float): RNA fraction [g/gDW].
            - D (float): DNA fraction [g/gDW].
            - L (float): Lipid fraction [g/gDW].

    Notes:
        - Applies water loss correction (−18 g/mol per monomer) for proteins, carbs, RNA, and DNA to account 
          for polymerization.
        - Lipids are handled using the 'lipid pseudoreaction - backbone' only (does not include tails).
    """

    # Extract stoichiometric matrix
    S, met_ids, rxn_ids = extractModelStoichMatrix(model)  

    # Initialize biomass fractions
    X = 0

    # Get each fraction
    P, X = getFraction(model, comps, 'P', X, S, met_ids, rxn_ids, water_correction=True)
    C, X = getFraction(model, comps, 'C', X, S, met_ids, rxn_ids, water_correction=True)
    R, X = getFraction(model, comps, 'R', X, S, met_ids, rxn_ids, water_correction=True)
    D, X = getFraction(model, comps, 'D', X, S, met_ids, rxn_ids, water_correction=True)
    L, X = getFraction(model, comps, 'L', X, S, met_ids, rxn_ids, water_correction=False)  

    # Find biomass reaction index
    try:
        bio_index = rxn_ids.index('r_4041')  # Need our biomass reaction ID*********************************************
    except ValueError:
        raise ValueError("Biomass reaction 'r_4041' not found in model.")

    # Add up any remaining components
    for i, met in enumerate(met_ids):
        matching_comp = next((c for c in comps if c[0] == met), None)
        if matching_comp:
            MW = matching_comp[1]
            abundance = -S[i, bio_index] * MW / 1000  # Convert to g/gDW
            X += abundance

    return X, P, C, R, D, L


def getFraction(model, comps, compType, X, S, met_ids, rxn_ids, water_correction):
    """
    Computes the g/gDW biomass fraction for a specific macromolecular component type.
    This is a helper function called by `sumBioMass()` to calculate the biomass contribution of
    a specific macromolecule type (e.g., protein, RNA) by summing the relevant metabolite
    abundances in the corresponding pseudoreaction.

    Parameters:
        model (cobra.Model): The COBRApy model.
        comps (list of tuple): Component metadata list: (metabolite ID, molecular weight, type).
        compType (str): One of ['P', 'C', 'R', 'D', 'L'] for protein, carbohydrate, RNA, DNA, lipid.
        X (float): Running total of biomass fraction.
        S (np.ndarray): Stoichiometric matrix from the model (dense).
        met_ids (list of str): List of metabolite IDs.
        rxn_ids (list of str): List of reaction IDs.
        water_correction (bool): Whether to subtract 18 g/mol from monomer MW for polymerization.

    Returns:
        tuple:
            - F (float): The computed biomass fraction for this component type [g/gDW].
            - X (float): Updated total biomass excluding lipids [g/gDW].

    Notes:
        - For lipids (compType == 'L'), this function uses the 'lipid pseudoreaction - backbone'.
        - Stoichiometry is assumed to be negative for reactants; mass = −(S * MW) / 1000.
    """

    # Define pseudoreaction name
    rxnNames = {
        'P': 'protein pseudoreaction',
        'C': 'carbohydrate pseudoreaction',
        'R': 'RNA pseudoreaction',
        'D': 'DNA pseudoreaction',
        'L': 'lipid pseudoreaction'
    }
    
    rxnName = rxnNames.get(compType)
    if not rxnName:
        raise ValueError(f"Invalid component type: {compType}")

    # Find reaction index
    try:
        fraction_index = rxn_ids.index(rxnName)
    except ValueError:
        raise ValueError(f"Reaction '{rxnName}' not found in model.")

    # Add up fraction
    F = 0
    if compType == 'L':
        # Lipid backbone special case
        try:
            lipid_index = rxn_ids.index('lipid pseudoreaction - backbone')
        except ValueError:
            raise ValueError("Reaction 'lipid pseudoreaction - backbone' not found in model.")

        # Sum up negative stoichiometry coefficients
        F = -np.sum(abs(S[:, lipid_index][S[:, lipid_index] < 0]))  # g/gDW
    else:
        # Other biomolecules
        componentMets = [c for c in comps if c[2] == compType]

        for i, met in enumerate(met_ids):
            matchingComp = next((c for c in componentMets if c[0] == met), None)
            if matchingComp:
                MW = matchingComp[1]
                MW_adjusted = MW - 18 if water_correction else MW  # Apply water loss correction
                abundance = -S[i, fraction_index] * MW_adjusted / 1000  # Convert to g/gDW
                F += abundance

    X += F
    return F, X

In [ ]:
tail_abundances = {
    "C10:0": 0.08,
    "C12:0": 0.10,
    "C14:0": 2.16,
    "C15:0": 0.58,
    "C16:0": 14.48,
    "C16:1": 24.12 + 18.06 + 36.59 + 2.24,  # if including both C16:1n9 and C16:1n7, etc etc
    "C16:1n9": 24.12,
    "C16:1n7": 18.06,
    "C16:1n6": 36.59,
    "C16:1n5": 2.24,
    "C16 unknown1": 0.3,
    "C16:2": 0.31,
    "C16:3": 0.44,
    "C18": 0.28,
    "C18:1n9": 0.18,
    "C18:1n7": 0.08
}

In [ ]:
lipid_definitions = {
    "PG 14:0-16:1": {
        "backbone": "phosphatidylglycerol",
        "backbone_id": "s_0089[c]",
        "tails": ["C14:0", "C16:1"],
        "abundance": {"BG8": 11.8}
    },
    "PG 16:1-16:1": {
        "backbone": "phosphatidylglycerol",
        "backbone_id": "s_0089[c]",
        "tails": ["C16:1", "C16:1"],
        "abundance": {"BG8": 17.1}
    },
    "PE 16:1-14:0": {
        "backbone": "phosphatidylethanolamine",
        "backbone_id": "s_0089[c]",
        "tails": ["C16:1", "C14:0"],
        "abundance": {"BG8": 1.2}
    },
    "PE 16:0-16:1": {
        "backbone": "phosphatidylethanolamine",
        "backbone_id": "s_0089[c]",
        "tails": ["C16:0", "C16:1"],
        "abundance": {"BG8": 12.4}
    },
    "PE 16:1-16:1": {
        "backbone": "phosphatidylethanolamine",
        "backbone_id": "s_0089[c]",
        "tails": ["C16:1", "C16:1"],
        "abundance": {"BG8": 19.0}
    },
    "PE 18:1-16:1": {
        "backbone": "phosphatidylethanolamine",
        "backbone_id": "s_0089[c]",
        "tails": ["C18:1", "C16:1"],
        "abundance": {"BG8": 12.5}
    },
    "PE 16:0-18:1": {
        "backbone": "phosphatidylethanolamine",
        "backbone_id": "s_0089[c]",
        "tails": ["C16:0", "C18:1"],
        "abundance": {"BG8": 7.0}
    }
}

In [ ]:
tail_aliases = [
    ("C10:0", "Decanoate", "Decanoic acid"),
    ("C12:0", "Laurate", "Lauric acid"),
    ("C14:0", "Myristate", "Myristic acid"),
    ("C15:0", "Pentadecanoate", "Pentadecanoic acid"),
    ("C16:0", "Palmitate", "Palmitic acid"),
    ("C16:1n9", "Palmitoleate (n-9 isomer)", "Palmitoleic acid (n-9 isomer)"),
    ("C16:1n7", "Palmitoleate", "Palmitoleic acid"),
    ("C16:1n6", "Hexadecenoate (n-6)", "Hexadecenoic acid (n-6)"),
    ("C16:1n5", "Hexadecenoate (n-5)", "Hexadecenoic acid (n-5)"),
    ("C16 unknown1", "Unknown C16 fatty acid", "none"),
    ("C16:2", "Hexadecadienoate", "Hexadecadienoic acid"),
    ("C16:3", "Hexadecatrienoate", "Hexadecatrienoic acid"),
    ("C18", "Stearate", "Stearic acid"),
    ("C18:1n9", "Oleate", "Oleic acid"),
    ("C18:1n7", "Vaccenat", "Vaccenic acid")
]